# Chương 14: Tự Động Hóa Quy Trình Học Máy với Pipelines

Có những quy trình tiêu chuẩn trong một dự án học máy có thể được tự động hóa. Trong Python scikit-learn, Pipelines giúp định nghĩa rõ ràng và tự động hóa các quy trình này. Trong chương này, bạn sẽ khám phá Pipelines trong scikit-learn và cách bạn có thể tự động hóa các quy trình học máy phổ biến.

## 14.1 Tự Động Hóa Quy Trình Học Máy

Có những quy trình tiêu chuẩn trong học máy ứng dụng. Tiêu chuẩn vì chúng khắc phục các vấn đề phổ biến như rò rỉ dữ liệu trong bộ kiểm thử của bạn. Python scikit-learn cung cấp tiện ích Pipeline để giúp tự động hóa các quy trình học máy. Pipelines hoạt động bằng cách cho phép một chuỗi tuyến tính các biến đổi dữ liệu được kết nối với nhau, kết thúc bằng một quá trình mô hình hóa có thể được đánh giá.

Mục tiêu là đảm bảo rằng tất cả các bước trong pipeline được giới hạn trong dữ liệu có sẵn cho việc đánh giá, chẳng hạn như tập dữ liệu huấn luyện hoặc từng fold của quy trình kiểm chứng chéo (cross validation).

## 14.2 Pipeline Chuẩn Bị Dữ Liệu và Mô Hình Hóa

Một lỗi dễ mắc phải trong học máy ứng dụng là rò rỉ dữ liệu từ tập dữ liệu huấn luyện sang tập dữ liệu kiểm tra. Để tránh cạm bẫy này, bạn cần một bộ kiểm thử mạnh mẽ với sự phân tách rõ ràng giữa huấn luyện và kiểm tra. Điều này bao gồm cả việc chuẩn bị dữ liệu.

Chuẩn bị dữ liệu là một cách dễ dàng để rò rỉ kiến thức về toàn bộ tập dữ liệu huấn luyện cho thuật toán. Ví dụ, chuẩn bị dữ liệu của bạn bằng cách chuẩn hóa (normalization) hoặc tiêu chuẩn hóa (standardization) trên toàn bộ tập dữ liệu huấn luyện trước khi học sẽ không phải là một bài kiểm tra hợp lệ vì tập dữ liệu huấn luyện sẽ bị ảnh hưởng bởi tỷ lệ của dữ liệu trong tập kiểm tra.

Pipelines giúp bạn ngăn chặn rò rỉ dữ liệu trong bộ kiểm thử của bạn bằng cách đảm bảo rằng việc chuẩn bị dữ liệu như tiêu chuẩn hóa được giới hạn trong từng fold của quy trình kiểm chứng chéo.

In [ ]:
# Tạo pipeline tiêu chuẩn hóa dữ liệu và sau đó tạo mô hình
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# tải dữ liệu
filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# tạo pipeline
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('lda', LinearDiscriminantAnalysis()))
model = Pipeline(estimators)

# đánh giá pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

## 14.3 Pipeline Trích Xuất Đặc Trưng và Mô Hình Hóa

Trích xuất đặc trưng là một quy trình khác dễ bị rò rỉ dữ liệu. Giống như chuẩn bị dữ liệu, các quy trình trích xuất đặc trưng phải được giới hạn trong dữ liệu trong tập dữ liệu huấn luyện của bạn.

Pipeline cung cấp một công cụ tiện dụng gọi là FeatureUnion cho phép kết quả của nhiều quy trình lựa chọn và trích xuất đặc trưng được kết hợp thành một tập dữ liệu lớn hơn trên đó một mô hình có thể được huấn luyện. Quan trọng là, tất cả việc trích xuất đặc trưng và hợp nhất đặc trưng xảy ra trong từng fold của quy trình kiểm chứng chéo.

In [ ]:
# Tạo pipeline trích xuất đặc trưng từ dữ liệu rồi tạo mô hình
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

# tải dữ liệu
filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# tạo feature union
features = []
features.append(('pca', PCA(n_components=3)))
features.append(('select_best', SelectKBest(k=6)))
feature_union = FeatureUnion(features)

# tạo pipeline
estimators = []
estimators.append(('feature_union', feature_union))
estimators.append(('logistic', LogisticRegression()))
model = Pipeline(estimators)

# đánh giá pipeline
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

## 14.4 Tóm Tắt

Trong chương này, bạn đã khám phá những khó khăn của rò rỉ dữ liệu trong học máy ứng dụng. Bạn đã khám phá các tiện ích Pipeline trong Python scikit-learn và cách chúng có thể được sử dụng để tự động hóa các quy trình học máy ứng dụng tiêu chuẩn. Bạn đã học cách sử dụng Pipelines trong hai trường hợp sử dụng quan trọng:

- Chuẩn bị dữ liệu và mô hình hóa giới hạn trong từng fold của quy trình kiểm chứng chéo.
- Trích xuất đặc trưng và hợp nhất đặc trưng giới hạn trong từng fold của quy trình kiểm chứng chéo.